In [2]:
import sys, os
from collections import OrderedDict
import numpy as np

sys.path.append('/Users/derek/projects/deep-learning-from-scratch')
from dataset.mnist import load_mnist

from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

편미분 구하는 법:

      x              x_out               out
     ---> (function) ------  ... ->  (prediction)
     dx               dx_out            dout
     
계산순서는 거꾸로
여러 브랜치로 갈릴 수 있음. 계산순서 안꼬이게 주의 해주어야 함. 그래프 이론에서 나오는 그리디 알고리즘이 쓰임.

사과그림으로.
문제: 물가를 낮추려먼 뭘 바꾸는게 가장 효과적일까?

In [3]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [6]:
apple = 100
apple_num = 2
tax = 1.1

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

before_tax = mul_apple_layer.forward(apple, apple_num)
total = mul_tax_layer.forward(before_tax, tax)

print(apple_final)

220.00000000000003


In [7]:
dtotal = 1  # baseline
dbefore_tax, dtax = mul_tax_layer.backward(dtotal)
dapple, dapple_num = mul_apple_layer.backward(dbefore_tax)

print("dtotal", dtotal)
print("dbefore_tax", dbefore_tax, "dtax", dtax)
print("dapple", dapple, "dapple_num", dapple_num)

dtotal 1
dbefore_tax 1.1 dtax 200
dapple 2.2 dapple_num 110.00000000000001


In [8]:
%%debug

mul_tax_layer.backward(dtotal)

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(3)<module>()

ipdb> s
--Call--
> <ipython-input-3-0b6f047a2549>(12)backward()
     10         return out
     11 
---> 12     def backward(self, dout):
     13         dx = dout * self.y
     14         dy = dout * self.x

ipdb> !dout
1
ipdb> !self.y
1.1
ipdb> self.x
200
ipdb> n
> <ipython-input-3-0b6f047a2549>(13)backward()
     11 
     12     def backward(self, dout):
---> 13         dx = dout * self.y
     14         dy = dout * self.x
     15         return dx, dy

ipdb> n
> <ipython-input-3-0b6f047a2549>(14)backward()
     11 
     12     def backward(self, dout):
     13         dx = dout * self.y
---> 14         dy = dout * self.x
     15         return dx, dy

ipdb> n
> <ipython-input-3-0b6f047a2549>(15)backward()
     11 
     12     def backward(self, dout):
     13         dx = dout * self.y
     14         dy = dout * self.x
---> 15         return dx, dy

ipdb> n
--Return--
(1.1, 200)
> <ipython-input-3

In [14]:
class Relu:
    def __init__(self):
        self.max = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask] = 0  # Bad. Nasty side effect !!
        dx = dout
        return dx

In [6]:
class ReluCorrect:
    def __init__(self):
        self.max = None
    
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dx = dout.copy()
        dx[self.mask] = 0
        return dx

Unwanted side effect example

In [7]:
x = np.array([1.0, -0.5])
relu = Relu()
y = relu.forward(x)

dy = np.ones(shape=y.shape)
print("dy", dy)
dx = relu.backward(dy)
print("dx", dx, "dy", dy)  # dy changed!

dy [1. 1.]
[1.]
[1.]
dx [1. 0.] dy [1. 0.]


In [8]:
class AddLayer:
    def forward(self, x, y):
        out = x + y
        return out
    
    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

    
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * ((1.0 - self.out) * self.out)  # Need to understand sigmoid derivative
                                                   # 도출이 쉽지 않은 함수라면 그래프 직접그려서 눈대중으로라도 확인해보기
        return dx

In [9]:
# Generally, if you understand scaler backprop well,
# tensor backprop is not hard to understand.

# In multiplication of scalar values, order doesen't matter.
# It does, however, in tensor computation.
# In linear algebra, x goes to left usually. In programming it's the opposite

x = np.random.rand(2)
print("x", x, "W", W, "y", y, sep='\n')
W = np.random.rand(2, 3)
y = np.dot(x, W)

x
[0.20235119 0.31279438]
W
[[0.96856771 0.61496447 0.79177973]
 [0.00726119 0.25114484 0.25910596]]
y
[0.19826209 0.20299549 0.24126446]


In [10]:
# to compute dot product of W and y, you need to transpose one of them
# Transposing W is better, becuase the result has the same shape as x

x = np.random.rand(3, 2)
W = np.random.rand(2, 5)
y = np.dot(x, W)
print("x", x, "W", W, "y", y, sep='\n')


print('\n', np.dot(W, y.T))
print('\n', np.dot(y, W.T))

x
[[0.2739352  0.22612145]
 [0.43627187 0.92762044]
 [0.6922855  0.71847096]]
W
[[0.62708551 0.04342238 0.69338254 0.70495486 0.4308865 ]
 [0.6709176  0.11300584 0.81869606 0.79995714 0.58366359]]
y
[[0.32348965 0.03744796 0.37506662 0.37399942 0.25001384]
 [0.89593665 0.12377049 1.0619425  1.04960856 0.72940194]
 [0.91615702 0.1112521  1.06822803 1.06277601 0.71764182]]

 [[0.8359267  2.35775179 2.37846158]
 [0.97343979 2.74986182 2.77082876]]

 [[0.8359267  0.97343979]
 [2.35775179 2.74986182]
 [2.37846158 2.77082876]]


In [11]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x, self.dw, self.db = (None,) * 3
    
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)  # For batch
        return dx
    
class SoftmaxLoss:
    def __init__(self):
        self.loss = None
        self.y, self.t = (None,) * 2
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

In [12]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size) 
        self.params['b2'] = np.zeros(output_size)

        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])

        self.lastLayer = SoftmaxLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x

    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1 :
            t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)  # This is wrong?

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        # forward
        self.loss(x, t)

        # backward
        dout = 1
        dout = self.lastLayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        grads = {}
        grads['W1'], grads['b1'] = self.layers['Affine1'].dW, self.layers['Affine1'].db
        grads['W2'], grads['b2'] = self.layers['Affine2'].dW, self.layers['Affine2'].db

        return grads

In [ ]:
# Train
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)

    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

In [ ]:
network.gradient(x_batch, t_batch)['W1'][:, 1]

In [ ]:
# Gradient check
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average( np.abs(grad_backprop[key] - grad_numerical[key]) )
    print(key + ":" + str(diff))

In [ ]:
grad_backprop[key], grad_numerical[key]